# Lesson 2 - Retrieval Augmented Generation (RAG)

### Import the Needed Packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

from datasets import load_dataset
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from tqdm.auto import tqdm
from DLAIUtils import Utils

import ast
import os
import pandas as pd

In [2]:
# get api key
utils = Utils()
PINECONE_API_KEY = utils.get_pinecone_api_key()

### Setup Pinecone

In [3]:
pinecone = Pinecone(api_key=PINECONE_API_KEY)

# Create a Pinecone index (vector DB)
utils = Utils()
INDEX_NAME = utils.create_dlai_index_name('dl-ai')
if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
  pinecone.delete_index(INDEX_NAME)

pinecone.create_index(name=INDEX_NAME, dimension=1536, metric='cosine',
  spec=ServerlessSpec(cloud='aws', region='us-west-2'))

index = pinecone.Index(INDEX_NAME)

### Load the Dataset

<p style="background-color:#fff1d7; padding:15px; "> <b>(Note: <code>max_articles_num = 500</code>):</b> To achieve a more comprehensive context for the Language Learning Model, a larger number of articles is generally more beneficial. In this lab, we've initially set <code>max_articles_num</code> to 500 for speedier results, allowing you to observe the outcomes faster. Once you've done an initial run, consider increasing this value to 750 or 1,000. You'll likely notice that the context provided to the LLM becomes richer and better. You can experiment by gradually raising this variable for different queries to observe the improvements in the LLM's contextual understanding.</p>

In [4]:
max_articles_num = 500
# This corpus has already been embedded
df = pd.read_csv('./data/wiki.csv', nrows=max_articles_num)
df.head()

,id,metadata,values
1,1-0,"{'chunk': 0, 'source': 'https://simple.wikiped...","[-0.011254455894231796, -0.01698738895356655, ..."
2,1-1,"{'chunk': 1, 'source': 'https://simple.wikiped...","[-0.0015197008615359664, -0.007858820259571075..."
3,1-2,"{'chunk': 2, 'source': 'https://simple.wikiped...","[-0.009930099360644817, -0.012211072258651257,..."
4,1-3,"{'chunk': 3, 'source': 'https://simple.wikiped...","[-0.011600767262279987, -0.012608098797500134,..."
5,1-4,"{'chunk': 4, 'source': 'https://simple.wikiped...","[-0.026462381705641747, -0.016362832859158516,..."


### Prepare the Embeddings and Upsert to Pinecone

In [5]:
prepped = []
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    meta = ast.literal_eval(row['metadata'])
    prepped.append({'id':row['id'], 
                    'values':ast.literal_eval(row['values']), 
                    'metadata':meta})
    if len(prepped) >= 250:
        index.upsert(prepped)
        prepped = []

  0%|          | 0/500 [00:00<?, ?it/s]

In [6]:
index.describe_index_stats()
# Check that the dimension is same as that of text-embedding-ada-002 (OpenAI)

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 500}},
 'total_vector_count': 500}

### Connect to OpenAI

In [7]:
OPENAI_API_KEY = utils.get_openai_api_key()
openai_client = OpenAI(api_key=OPENAI_API_KEY)

def get_embeddings(articles, model="text-embedding-ada-002"):
   return openai_client.embeddings.create(input = articles, model=model)

### Run Your Query

In [8]:
query = "What is the Babylon gate?"

embed = get_embeddings([query])
res = index.query(vector=embed.data[0].embedding, top_k=3, include_metadata=True)
text = [r['metadata']['text'] for r in res['matches']]
print('\n'.join(text))

Ancient history 

Before the middle of the sixth century BCE, Afghanistan was held by the Medes.
Then the Achaemenids took over control of the land and made it part of the Persian empire. Alexander the great defeated and conquered the Persian Empire in 330 BCE. He founded some cities in the area. The people used Macedonian culture and language. After Alexander, Greco-Bactrians, Scythians, Kushans, Parthians and Sassanians ruled the area.

Kushans spread Buddhism from India in the 1st century BCE, and Buddhism remained an important religion in the area until the Islamic conquest in the 7th century CE.

The Buddhas of Bamiyan were giant statues, a reminder of Buddhism in Afghanistan. They were destroyed by the Taliban in 2001. There were international protests. The Taliban believe that the ancient statues were un-Islamic and that they had a right to destroy them.

Medieval history 

Arabs introduced Islam in the 7th century and slowly began spreading the new religion. In the 9th and 10th

### Build the Prompt

In [10]:
query = "write an article titled: what is the Babylon gate?"
embed = get_embeddings([query])
res = index.query(vector=embed.data[0].embedding, top_k=3, include_metadata=True)

contexts = [
    x['metadata']['text'] for x in res['matches']
]

prompt_start = (
    "Answer the question based on the context below.\n\n"+
    "Context:\n"
)

prompt_end = (
    f"\n\nQuestion: {query}\nAnswer:"
)

prompt = (
    prompt_start + "\n\n---\n\n".join(contexts) + 
    prompt_end
)

print(prompt)

Answer the question based on the context below.

Context:
Now, 150 years later, it really is a big city.

In modern times many cities have grown bigger and bigger. The whole area is often called a  "metropolis"  and can sometimes include several small ancient towns and villages. The metropolis of London includes London, Westminster, and many old villages such as Notting Hill, Southwark, Richmond, Greenwich, etc. The part that is officially known as the " City of London " only takes up one square mile. The rest is known as "Greater London. " Many other cities have grown in the same way.

These giant cities can be exciting places to live, and many people can find good jobs there, but modern cities also have many problems. Many people cannot find jobs in the cities and have to get money by begging or by crime. Automobiles, factories, and waste create a lot of pollution that makes people sick.

Urban history 

Urban history is history of civilization. The first cities were made in ancient 

### Get the Summary 

In [11]:
res = openai_client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    temperature=0,
    max_tokens=636,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
print('-' * 80)
print(res.choices[0].text)

--------------------------------------------------------------------------------

The Babylon Gate, also known as the Ishtar Gate, was a monumental entrance to the ancient city of Babylon, located in present-day Iraq. It was built during the reign of King Nebuchadnezzar II in the 6th century BCE and was considered one of the most impressive architectural achievements of the time.

The gate was part of a larger project to rebuild the city of Babylon, which had been destroyed by the Assyrians. It was constructed using glazed bricks, with intricate designs and images of animals and gods adorning its walls. The gate was dedicated to the goddess Ishtar, the patron deity of Babylon, and was meant to impress and intimidate visitors with its grandeur and power.

The Babylon Gate was not only a symbol of the city's wealth and strength, but it also served as a defensive structure. The gate was part of a larger wall that surrounded the city, and its imposing size and design were meant to deter an

In [12]:
# Try on Mistral 7B model (offline)
from llama_cpp import Llama

model_path = "/Users/trucvietle/Downloads/llm-models/mistral-7b-instruct-v0.1.Q6_K.gguf"
llm = Llama(model_path=model_path,
            n_ctx=8192, n_batch=512,
            n_threads=7, n_gpu_layers=2,
            verbose=False, seed=42)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/trucvietle/Downloads/llm-models/mistral-7b-instruct-v0.1.Q6_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader:

In [13]:
output = llm(prompt, echo=True, stream=False, max_tokens=4096)

KeyboardInterrupt: 